In [10]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
import seaborn as sns

np.set_printoptions(threshold=np.inf)

ModuleNotFoundError: No module named 'seaborn'

In [11]:
raw_data = pd.read_csv('data/train.csv')

C:\Users\Ethan\Anaconda3\envs\DS\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
raw_data.columns

Index(['GameId', 'PlayId', 'Team', 'X', 'Y', 'S', 'A', 'Dis', 'Orientation',
       'Dir', 'NflId', 'DisplayName', 'JerseyNumber', 'Season', 'YardLine',
       'Quarter', 'GameClock', 'PossessionTeam', 'Down', 'Distance',
       'FieldPosition', 'HomeScoreBeforePlay', 'VisitorScoreBeforePlay',
       'NflIdRusher', 'OffenseFormation', 'OffensePersonnel',
       'DefendersInTheBox', 'DefensePersonnel', 'PlayDirection', 'TimeHandoff',
       'TimeSnap', 'Yards', 'PlayerHeight', 'PlayerWeight', 'PlayerBirthDate',
       'PlayerCollegeName', 'Position', 'HomeTeamAbbr', 'VisitorTeamAbbr',
       'Week', 'Stadium', 'Location', 'StadiumType', 'Turf', 'GameWeather',
       'Temperature', 'Humidity', 'WindSpeed', 'WindDirection'],
      dtype='object')

In [7]:
BUFFER = 60
player_count = np.zeros((BUFFER*2+1,BUFFER*2+1))
mean_yards = np.zeros((BUFFER*2+1,BUFFER*2+1))
for _,play in raw_data.groupby(['PlayId']):
    yards = play['Yards'].iloc[0]
    ox = play.loc[play['NflId'] == play['NflIdRusher'],'X'].iloc[0]
    oy = play.loc[play['NflId'] == play['NflIdRusher'],'Y'].iloc[0]
    for _,player in play.iterrows():
        x = int(round(player['X']-ox+BUFFER))
        y = int(round(player['Y']-oy+BUFFER))
        player_count[y,x] += 1
        mean_yards[y,x] = mean_yards[y,x] + (yards - mean_yards[y,x]) / player_count[y,x]
mean_yards -= raw_data['Yards'].mean()

NameError: name 'sns' is not defined

In [9]:
fig = sns.heatmap(mean_yards)
fig.show()

NameError: name 'sns' is not defined